In [5]:
import os
from libs.CeleryTasks import tasks
from libs.json import JSON

In [7]:
import time

In [2]:
group = '关注词'
result = tasks.p4p_record.apply_async(kwargs={'group': group}, queue='celery')

In [2]:
result = tasks.osoeco_checkin.apply_async(queue='celery')

In [11]:
result.ready()

True

In [22]:
result.get(timeout=1)

In [5]:
market_eyelash = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
market_tools = JSON.deserialize('.', 'storage', 'markets.json')['Tools']

. storage markets.json
.\storage\markets.json was deserialized!
. storage markets.json
.\storage\markets.json was deserialized!


In [ ]:
result = tasks.init.delay(market_eyelash)

In [ ]:
result.ready()

In [ ]:
result.get(timeout=1)

In [ ]:
result.get(timeout=1)

In [ ]:
result

In [ ]:
os.getpid()

In [3]:
group = '直通车App'

In [5]:
result = tasks.p4p_check.apply_async(kwargs={'group': group}, queue='Eyelashes_p4p')
result = tasks.p4p_check.apply_async(kwargs={'group': '0直通车'}, queue='Tools_p4p')

In [6]:
result = tasks.p4p_turn_all_off.apply_async(kwargs={'group': group}, queue='Eyelashes_p4p')
result = tasks.p4p_turn_all_off.apply_async(kwargs={'group': '0直通车'}, queue='Tools_p4p')

In [ ]:
result.ready()

In [ ]:
result.get()

In [ ]:
result = tasks.inquiry_check.apply_async(queue='Eyelashes_inquiry')

In [ ]:
result = tasks.p4p_info.apply_async(queue='Eyelashes_p4p')

In [ ]:
result.ready()

In [ ]:
result.get()

In [ ]:
from celery.schedules import crontab
from celery import Celery

In [ ]:
app = Celery('scheduler')
app.config_from_object('celeryconfig')
app.conf.beat_schedule = {
    'Eyelashes_p4p_info': {
    'task': 'tasks.p4p_info',
    'schedule': crontab(minute='*'),
#     'kwargs': {'group': '0直通车'},
    'options': {'queue': 'Eyelashes_p4p'}
    }
}

In [14]:
result = tasks.crawl_product_ranking.apply_async(kwargs={'keyword': 'mink eyelash', 'pages': 5}, queue='products_ranking')

In [16]:
result.ready()

True

In [17]:
records = result.get()

In [18]:
len(records)

2

In [19]:
print(records['datetime'], len(records['records']))
records['records'][36]

2018-10-21 23:01:11 180


{'location': {'page': 2, 'position': 1},
 'is_top_sponsor': False,
 'is_sponsor': False,
 'product': {'id': '60803158879',
  'title': '5D mink eyelash high quality mink eyelashes',
  'href': '//www.alibaba.com/product-detail/5D-mink-eyelash-high-quality-mink_60803158879.html',
  'img': '//sc01.alicdn.com/kf/HTB1YreEXhrvK1RjSszeq6yObFXaX/5D-mink-eyelash-high-quality-mink-eyelashes.jpg_300x300.jpg'},
 'company': {'name': 'Qingdao Goodyard International E-Commerce Co., Ltd.',
  'href': '//goodyardeyelash.en.alibaba.com/company_profile.html#top-nav-bar',
  'years': '1',
  'transaction': {'counts': '', 'volume': '', 'level': 3.0},
  'response_rate': '98.7%'}}

In [ ]:
result = tasks.crawl_product_ranking.apply_async(kwargs={'keyword': 'mink eyelash', 'pages': 5}, queue='products_ranking')


In [14]:
def crawl_products_rankings(kws, max_processes=3):
    processes_count = 0
    results = {}
    idx_kws = -1
    while True:
        while processes_count < max_processes and (idx_kws+1) < len(kws):
            idx_kws +=1
            kw = kws[idx_kws]
            kwargs={'keyword': kw, 'pages': 5}
            result = tasks.crawl_product_ranking.apply_async(kwargs=kwargs, queue='products_ranking')
            results[kw] = result
            processes_count += 1
            
            print(kw, end=',')

        print()
        print('--------------')

        if processes_count == 0:
            break

        records = []
        finished_kws = []
        while True:
            for kw in results:
                result = results[kw]
                if result.ready():
                    record = result.get()
                    record['keyword'] = kw
                    records.append(record)
                    finished_kws.append(kw)
                    processes_count -= 1
                    
            for kw in finished_kws:
                results.pop(kw)
            if records:
                print('finished:', len(records), finished_kws)
                break
            else:
                time.sleep(1)

In [15]:
kws = ['eyelash', 'lash', 'mink eyelash', 'wholesale eyelash', 'eyelash vendor']
crawl_products_rankings(kws)

eyelash,lash,mink eyelash,
--------------
finished: 1 ['lash']
wholesale eyelash,
--------------
finished: 2 ['eyelash', 'mink eyelash']
eyelash vendor,
--------------
finished: 1 ['eyelash vendor']

--------------
finished: 1 ['wholesale eyelash']

--------------
